# Preliminary Experiment
To begin exploring the relationships in our data we decided to apply k-means clustering analysis. 

In [20]:
# Imports
import pandas as pd
from pandas.plotting import scatter_matrix
pd.set_option('display.max_columns', None)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sklearn as skl
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn import set_config
set_config(transform_output="pandas")
from IPython.display import display
%matplotlib inline

SEED = 42
np.random.seed(SEED)

Loading the cleaned data

In [21]:
data = pd.read_csv("data/merged_data_clean.csv")
data.head()

,County,Households Served,Individuals Served,Pounds of Food Distributed,Children Served via non-federal child nutrition programs,Pounds of food distributed via non-federal child nutrition progr,Virginia City/County Boundaries,LAT,LON,Pop2010,OHU2010,NUMGQTRS,LAPOP1_10,LAPOP05_10,LAPOP1_20,LALOWI1_10,LALOWI05_10,LALOWI1_20,lapophalf,lalowihalf,lakidshalf,laseniorshalf,lawhitehalf,lablackhalf,laasianhalf,lanhopihalf,laaianhalf,laomultirhalf,lahisphalf,lahunvhalf,lasnaphalf,lapop1,lalowi1,lakids1,laseniors1,lawhite1,lablack1,laasian1,lanhopi1,laaian1,laomultir1,lahisp1,lahunv1,lasnap1,lapop10,lalowi10,lakids10,laseniors10,lawhite10,lablack10,laasian10,laaian10,laomultir10,lahisp10,lahunv10,lasnap10,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP,PCTGQTRS,PovertyRate,MedianFamilyIncome,Urban,LILATracts_halfAnd10,LowIncomeTracts,LA1and10,LAhalfand10,LATracts_half,lapophalfshare,lalowihalfshare,lakidshalfshare,laseniorshalfshare,lawhitehalfshare,lablackhalfshare,laasianhalfshare,lanhopihalfshare,laaianhalfshare,laomultirhalfshare,lahisphalfshare,lahunvhalfshare,lasnaphalfshare,lapop1share,lalowi1share,lakids1share,laseniors1share,lawhite1share,lablack1share,laasian1share,lanhopi1share,laaian1share,laomultir1share,lahisp1share,lahunv1share,lasnap1share
0,Accomack County,21391.0,62975.0,1248586.00,3112.0,42929.0,432.0,37.765943,-75.757807,33164.0,13798.0,428.0,727.0,727.0,0.0,298.0,298.0,0.0,30794.0,13236.0,6482.0,5812.0,19984.0,8864.0,164.0,36.0,124.0,1626.0,2574.0,1397.0,1854.0,27105.0,11970.0,5731.0,5089.0,17549.0,7832.0,147.0,30.0,116.0,1431.0,1926.5,1274.0,1681.0,727.0,298.0,135.0,156.0,714.0,4.0,3.0,0.0,6.0,2.0,45.0,23.0,14271.0,6925.0,6336.0,21662.0,9303.0,183.0,40.0,135.0,1841.0,2850.0,1352.0,1920.0,0.922222,17.811111,54276.875000,0.0,0.0,1.0,0.0,0.0,0.0,93.960000,35.898889,19.612222,15.794444,66.442222,22.252222,0.436667,0.090000,0.332222,4.408889,6.450469,8.854444,12.151111,83.995556,32.641111,17.704444,13.681111,59.306667,19.977778,0.393333,0.070000,0.308889,3.928889,4.670000,8.121111,11.140000
1,Albemarle County,29526.0,78415.0,2044282.70,857.0,18305.0,1464.0,38.024184,-78.553506,98970.0,38157.0,5010.5,11786.0,30767.0,11450.0,3490.0,8892.0,3379.0,79776.0,17393.0,17712.0,11204.0,66334.0,6614.0,1203.0,35.0,194.0,3575.0,3886.5,983.0,1272.0,56224.0,10960.0,13245.0,8423.0,48600.0,3929.0,543.0,21.0,124.0,1909.5,1926.5,391.0,763.0,336.0,111.0,60.0,62.0,281.0,41.0,2.0,0.0,12.0,17.0,2.0,9.0,21693.0,21285.0,14124.0,79738.0,9600.0,1448.0,50.5,252.0,4703.0,5330.0,1887.0,1844.0,10.509091,14.681818,98792.666667,0.0,0.0,0.0,0.0,0.0,0.0,80.623636,19.180909,16.820000,11.156818,66.632727,6.759545,2.360000,0.035000,0.196364,3.680909,4.082273,5.490455,3.234091,63.318500,12.608500,14.641500,9.564500,54.430000,4.594000,1.264872,0.023500,0.146000,2.497500,2.781000,1.136000,2.292500
2,Alleghany County,4393.0,11912.0,312425.78,0.0,0.0,180.0,37.787905,-80.008669,16250.0,6891.0,281.0,2664.0,3797.0,1914.0,1283.0,1855.0,1067.0,15315.0,5838.0,3255.0,3021.0,14323.0,679.0,33.0,11.0,29.0,238.0,167.0,419.0,848.0,13284.0,4864.0,2785.0,2666.0,12633.0,420.0,20.0,6.0,27.0,178.0,133.0,341.0,677.0,750.0,216.0,147.0,146.0,737.0,3.0,0.0,0.0,10.0,6.0,7.0,18.0,5780.0,3450.0,3305.0,15145.0,761.0,37.0,11.0,29.0,267.0,176.0,424.0,909.0,1.553333,18.433333,63322.833333,0.0,0.0,1.0,0.0,0.0,0.0,95.991667,34.418333,20.260000,18.801667,90.663333,3.610000,0.180000,0.053333,0.155000,1.328333,0.971667,5.088333,11.250000,85.690000,29.541667,17.840000,17.011667,82.010000,2.348333,0.120000,0.031667,0.143333,1.031667,0.818333,4.220000,9.371667
3,Amelia County,1762.0,5334.0,123632.00,0.0,0.0,1224.0,37.331927,-77.977462,12690.0,4821.0,128.0,2089.0,2089.0,0.0,781.0,781.0,0.0,12646.0,3970.0,2813.0,1994.0,9308.0,2913.0,27.0,4.0,56.0,338.0,289.0,163.0,512.0,12441.0,3910.0,2770.0,1968.0,9182.0,2837.0,27.0,4.0,56.0,335.0,287.0,163.0,503.0,2089.0,781.0,429.0,292.0,1438.0,265.0,5.0,10.0,47.5,40.0,15.0,86.0,

In [22]:
print("# entries: ", data.shape[0])
print("# features: ", data.shape[1])
print("# categorical features: ", len(data.select_dtypes(include=['object']).columns.tolist()))
print("List categorical features: ", data.select_dtypes(include=['object']).columns.tolist())
print("# missing values: ", data.isnull().sum().sum())
desc_data = data.describe()
desc_data

# entries:  129
# features:  103
# categorical features:  1
List categorical features:  ['County']
# missing values:  0


,Households Served,Individuals Served,Pounds of Food Distributed,Children Served via non-federal child nutrition programs,Pounds of food distributed via non-federal child nutrition progr,Virginia City/County Boundaries,LAT,LON,Pop2010,OHU2010,NUMGQTRS,LAPOP1_10,LAPOP05_10,LAPOP1_20,LALOWI1_10,LALOWI05_10,LALOWI1_20,lapophalf,lalowihalf,lakidshalf,laseniorshalf,lawhitehalf,lablackhalf,laasianhalf,lanhopihalf,laaianhalf,laomultirhalf,lahisphalf,lahunvhalf,lasnaphalf,lapop1,lalowi1,lakids1,laseniors1,lawhite1,lablack1,laasian1,lanhopi1,laaian1,laomultir1,lahisp1,lahunv1,lasnap1,lapop10,lalowi10,lakids10,laseniors10,lawhite10,lablack10,laasian10,laaian10,laomultir10,lahisp10,lahunv10,lasnap10,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP,PCTGQTRS,PovertyRate,MedianFamilyIncome,Urban,LILATracts_halfAnd10,LowIncomeTracts,LA1and10,LAhalfand10,LATracts_half,lapophalfshare,lalowihalfshare,lakidshalfshare,laseniorshalfshare,lawhitehalfshare,lablackhalfshare,laasianhalfshare,lanhopihalfshare,laaianhalfshare,laomultirhalfshare,lahisphalfshare,lahunvhalfshare,lasnaphalfshare,lapop1share,lalowi1share,lakids1share,laseniors1share,lawhite1share,lablack1share,laasian1share,lanhopi1share,laaian1share,laomultir1share,lahisp1share,lahunv1share,lasnap1share
count,129.000000,129.000000,1.290000e+02,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.00000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.00000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000
mean,10442.325581,31447.705426,6.752746e+05,799.259690,11578.108527,936.434109,37.526000,-78.551888,39785.736434,15824.751938,1370.042636,7439.364341,14324.984496,6221.379845,2154.403101,4252.131783,1757.046512,34289.565891,9809.674419,7346.441860,4983.798450,26120.744186,5224.279070,360.263566,14.224806,108.550388,1113.782946,1168.903101,720.860465,1351.054264,23122.813953,6610.236434,5082.864341,3512.069767,18594.034884,3760.643411,177.751938,8.767442,77.403101,635.740310,630.341085,440.790698,886.558140,758.941860,221.457364,150.147287,130.736434,584.317829,62.651163,1.709302,2.403101,12.011628,10.550388,11.72093,26.922481,12083.434109,8549.015504,5971.589147,30758.531008,6262.813953,441.03876,16.197674,132.178295,1434.868217,1544.488372,895.534884,1615.782946,3.649403,14.463818,70180.666564,0.364341,0.271318,0.620155,0.286822,0.503876,0.356589,86.385081,27.692074,18.155039,13.698033,65.537811,16.075564,0.820913,0.033799,0.249180,2.660698,2.725202,5.118815,9.894082,67.789233,21.370299,14.265991,11.066008,52.920615,11.827629,0.484522,0.022247,0.193238,1.818989,1.799169,3.531992,7.574178
std,9164.694565,28350.674849,6.380606e+05,1184.974626,17472.897261,609.911108,0.680862,1.739062,35223.856365,14275.178774,1521.043718,8073.283321,16150.064371,7695.382160,2346.325270,4590.878381,2220.130292,30458.703576,7664.433740,6611.499513,3924.610593,23151.694703,5463.431733,440.886563,16.594069,104.478613,1160.484220,1301.052702,607.854896,1103.014083,18668.321903,5026.453690,4342.826004,2697.100185,16394.434685

Setting up a sklearn pipeline

Conducting clustering 